# 브런치 글 추천 샘플 프로그램

용량 관계상 데이터는 주피터 노트북 내의 샘플을 확인하는 것으로 대체하였습니다.

## 추천 방법

각 독자 별로 아직 읽지 않은 글을 추려낸 다음, 각 글마다 100점 만점으로 점수를 부여하여 높은 순서대로 추천하였습니다.
점수 구성은 다음과 같습니다.

1) 최신성 : 글이 발행된 시점에 따라 가장 최신의 글에 30점, 가장 오래된 글에 0점을 부여하고 그 사이는 등간격으로 점수 배분. 시간 구분은 일 단위까지만 하고, 동일한 날에 작성된 글은 평균치를 적용

유사성 및 인기도 : 총 70점을 다음과 같이 분배하여 계산.

2) 유사성 : 각 독자 별로 그 동안 읽은 글 중 팔로잉하는 작가의 글의 비율을 '구독 강도'로 정의하고, 구독 강도가 높을수록 유사한 성격의 글을 더 많이 읽는다고 가정. 70점 줌 구독 강도의 비중만큼을 유사성에 배분하고 나머지를 인기도에 배분. 유사성은 전체 텍스트의 제목을 tf-idf로 분석하여 벡터화한 행렬에서 해당 독자가 읽은 글의 제목 벡터의 합의 평균을 구한 후, 그 평균 벡터와 전체 제목 벡터 간의 코사인 유사도를 구하여 그 유사도가 높은 순서대로 점수를 배분. 가장 높은 텍스트에 만점을, 가장 낮은 텍스트에 0점을 부여하고 그 사이는 등간격으로 점수 배분. 팔로잉하는 작가가 있을 경우 팔로잉하는 작가가 쓴 글과 그렇지 않은 글을 구분하여 각각 점수를 매긴 후 구독 강도에 따라 각 그룹별로 추천하는 개수를 분배

3) 인기도 : 단순한 조회수 합산 순위만으로는 '앞으로 읽음직한 글'을 추천하는 본 대회에 활용하기 어려울 것으로 판단하여 각 월별 조회수를 구하여 각각 가중치를 곱해 합한 값으로 순위를 매김. 이 때 데이터 상의 마지막 달인 2019년 2월은 다시 반으로 나누어 조회수를 계산. 총 6그룹으로 구분됨. 가중치는 1, 2, 3, 4, 5, 6의 제곱으로 부여.


In [1]:
import pickle
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize
from time import time

In [2]:
#loading userwise history of articles. data type : python dictionary
with open("userwiseReadHistory.pkl", 'rb') as f:
    userwiseReadHistory = pickle.load(f)

In [11]:
# sample form of elements of userwiseReadHistory
print(list(userwiseReadHistory.keys())[:3])
print(userwiseReadHistory['#e208be4ffea19b1ceb5cea2e3c4dc32c'])

['#e208be4ffea19b1ceb5cea2e3c4dc32c', '#0a3d493f3b2318be80f391eaa00bfd1c', '#b90d3ee7ed0d7d827aae168e159749f1']
['@kty0613_91', '@svillustrated_43', '@svillustrated_44', '@jericho27_410', '@enamoramiento87_277', '@dapls_238', '@dapls_249', '@clncompany_168', '@ida0724_22', '@ida0724_11', '@alexkang_905', '@alexkang_900', '@yunta2_844', '@finance1026_211', '@alexkang_910', '@peregrino97_826', '@mobiinside_1281', '@alexkang_913', '@roysday_262', '@jordan777_2328', '@jordan777_2328', '@jordan777_2333', '@fishz_354', '@wndud961_49', '@newslaw_14', '@ysp0722_1128', '@ge999_14', '@chacha0401_45', '@flowerdog_370', '@kihyonkim_10', '@lois2054_21', '@nasu85_118', '@bestkkim_35', '@bbreeze_20', '@ib-77vision_19', '@trost_86', '@commsm_14', '@jordan777_2339', '@dkam_3', '@dsp9596_449', '@sylee118_39', '@alexkang_919', '@alexkang_918', '@alexkang_916', '@toriteller_360', '@dailylife_388', '@dailylife_390', '@mobiinside_1297', '@clncompany_181', '@pomnyun_87', '@lvvmvv_68', '@ihearyou_66', '@beich

In [12]:
# loading metadata of articles. type : pandas DataFrame
with open("metaDataForRecomm.pkl", 'rb') as f:
    metaDataForRecomm = pickle.load(f)

In [44]:
# sample form of metaDataForRecomm
print(metaDataForRecomm.shape)
metaDataForRecomm.head()

(629252, 3)


,articleId,datetime,writer
0,@bookdb_782,2016-09-27 11:47:07,@bookdb
1,@kohwang56_81,2016-05-13 07:39:09,@kohwang56
2,@hannahajink_4,2015-11-20 14:28:07,@hannahajink
3,@bryceandjuli_88,2017-04-01 22:59:21,@bryceandjuli
4,@mijeongpark_34,2018-04-10 01:55:42,@mijeongpark


In [15]:
# loading followers' list of users
with open('userFollowers.pkl', 'rb') as f:
    userFollowers = pickle.load(f)

In [19]:
# sample form of elements of userwiseReadHistory. type : python dictionary
print(list(userFollowers.keys())[:3])
print(userFollowers['#e208be4ffea19b1ceb5cea2e3c4dc32c'])

['#901985d8bc4c481805c4a4f911814c4a', '#1fd89e9dcfa64b45020d9eaca54e0eed', '#1d94baaea71a831e1f33e1c6bd126ed5']
['@kecologist' '@darunwrite' '@investup' '@travie' '@hiwriting'
 '@svillustrated' '@gilpark227' '@slowworks' '@holidaymemories' '@supims'
 '@thecapitalist' '@conceptnd' '@little-prince' '@alkony' '@gilberthan'
 '@jasmin2016' '@sakiroo' '@jerad' '@tetye3' '@alexkang' '@montb'
 '@julieted17' '@imagineer' '@londoncalling' '@mobiinside' '@yeowoonbae'
 '@roysday' '@phdcandidate' '@ida0724' '@tjkmix' '@parkisthinking'
 '@yundaehyupjun' '@mirboristar' '@editking' '@comeintothe' '@peregrino97'
 '@jaeholee' '@gisoopa' '@mignoncat' '@syoooons' '@clncompany'
 '@jeeyeonkim' '@12m17d' '@pomnyun' '@dong02' '@dailylife' '@5bro'
 '@yjjang' '@growingzero' '@slist' '@meanimize' '@vislakr'
 '@onestepculture' '@fermata' '@ebprux' '@andrewyhc' '@eomtank' '@ajtogg'
 '@qeemche' '@bestar' '@ideawriter' '@yujaechun' '@platum' '@slowdiamond'
 '@bzup' '@topmage' '@jordan777' '@toriteller' '@tschoe56' '

In [22]:
# 각 글의 제목과 부제목 텍스트로 tf-idf 방식의 벡터 맵핑. unigram 및 bigram을 활용하여 상위 60000개 토큰을 맵핑에 활용
# type : scipy sparse matrix
# 텍스트 전체 내용을 바탕으로 doc2vec 방법을 활용하여 맵핑해 보았으나,
# 제목 및 부제목만을 활용한 tf-idf 맵핑보다 메인 소재를 잡아내는데 부족한 퍼포먼스를 보여서 활용하지 않았음.

with open("titleTfidfMatrix.pkl", 'rb') as f:
    titleTfidfMatrix = pickle.load(f)

In [23]:
titleTfidfMatrix

<643104x60000 sparse matrix of type '<class 'numpy.float64'>'
	with 3435023 stored elements in Compressed Sparse Row format>

In [20]:
# detailed data for each user. type : python dictionary
with open("testUsersDict.pkl", 'rb') as f:
    testUsersDict = pickle.load(f)

In [21]:
# keys : userid 
# values : numpy array of four elements : portion of the number of articles written by following writers, 
#                                            the number of articles written by following writers, 
#                                            the number of articles not written by following writers,
#                                            the reader vector of the user

testUsersDict

{'#7ee14df8642a7925b1465ff5c89efe5b': array([0.18691588785046728, 20.0, 107.0,
        <1x60000 sparse matrix of type '<class 'numpy.float64'>'
 	with 342 stored elements in Compressed Sparse Row format>], dtype=object),
 '#8420b9385b282028eebf1ad6b4a221c0': array([0.056112224448897796, 28.0, 499.0,
        <1x60000 sparse matrix of type '<class 'numpy.float64'>'
 	with 831 stored elements in Compressed Sparse Row format>], dtype=object),
 '#c9b31d8b64357f5854b1ba55b32eb6d3': array([0.009708737864077669, 1.0, 103.0,
        <1x60000 sparse matrix of type '<class 'numpy.float64'>'
 	with 502 stored elements in Compressed Sparse Row format>], dtype=object),
 '#9bb1e13b5481fa3737af20870b25c723': array([0.08585630366018979, 190.0, 2213.0,
        <1x60000 sparse matrix of type '<class 'numpy.float64'>'
 	with 5095 stored elements in Compressed Sparse Row format>], dtype=object),
 '#37d5f99a7f12c9ba90c4e2ac92e54ab6': array([0.5913978494623656, 55.0, 93.0,
        <1x60000 sparse matrix of t

(recall) 인기도 계산 방식

단순한 조회수 합산 순위만으로는 '앞으로 읽음직한 글'을 추천하는 본 대회에 활용하기 어려울 것으로 판단하여 각 월별 조회수를 구하여 각각 가중치를 곱해 합한 값으로 순위를 매김. 이 때 데이터 상의 마지막 달인 2019년 2월은 다시 반으로 나누어 조회수를 계산. 총 6그룹으로 구분됨.
가중치는 1, 2, 3, 4, 5, 6의 제곱으로 부여.

In [24]:
# loading popularity data. data type : python list

with open("popularityOrdered.pkl", 'rb') as f:
    popularityOrdered = pickle.load(f)

In [26]:
popularityOrdered[:5]

[('@brunch_151', [0, 0, 0, 0, 0, 66903]),
 ('@sweetannie_145', [0, 0, 0, 1157, 11958, 4696]),
 ('@intlovesong_28', [1879, 1242, 2967, 9644, 3105, 4923]),
 ('@tenbody_1305', [5457, 4404, 4139, 5867, 3436, 5619]),
 ('@conbus_43', [274, 168, 194, 8028, 5702, 4067])]

In [30]:
# data of the indices of articles in metaDataForRecomm which is a DataFrame of all articles. type : python dictionary
with open("titleDict.pkl", 'rb') as f:
    titleDict = pickle.load(f)

In [35]:
print(len(titleDict))
titleDict

643104


{'@bookdb_782': 0,
 '@kohwang56_81': 1,
 '@hannahajink_4': 2,
 '@bryceandjuli_88': 3,
 '@mijeongpark_34': 4,
 '@jordan777_1334': 5,
 '@m-life_283': 6,
 '@hohogg_131': 7,
 '@elang8151_229': 8,
 '@purnbd_9': 9,
 '@hukho_247': 10,
 '@m-life_288': 11,
 '@purnbd_7': 12,
 '@mignoncat_41': 13,
 '@seesawyou_130': 14,
 '@sk0279_404': 15,
 '@fskene_247': 16,
 '@byulkim_101': 17,
 '@roke_70': 18,
 '@haninorway19_103': 19,
 '@dj0999_19': 20,
 '@seesawyou_133': 21,
 '@poldee_115': 22,
 '@kotani4_299': 23,
 '@brunchejej_22': 24,
 '@jerad_200': 25,
 '@seung4298_4': 26,
 '@jordan777_1333': 27,
 '@sharppen01_25': 28,
 '@pencil-k_478': 29,
 '@glassdice_20': 30,
 '@swkyung0221_3': 31,
 '@yooym2000_168': 32,
 '@hanaru_53': 33,
 '@melomane_50': 34,
 '@hyunj2407_14': 35,
 '@ubermensch_20': 36,
 '@slist_84': 37,
 '@sunyoule_41': 38,
 '@eunbinkim_70': 39,
 '@eunmikim_145': 40,
 '@byulkim_102': 41,
 '@leedalhee_65': 42,
 '@skyzakard_73': 43,
 '@ubermensch_23': 44,
 '@seunghyehan_53': 45,
 '@amberjeon48_292': 4

In [36]:
# IDs of target users
with open("predict/test.users", 'r') as f:
    testUsers = f.read().strip().split()

In [38]:
testUsers[:5]

['#7ee14df8642a7925b1465ff5c89efe5b',
 '#8420b9385b282028eebf1ad6b4a221c0',
 '#c9b31d8b64357f5854b1ba55b32eb6d3',
 '#9bb1e13b5481fa3737af20870b25c723',
 '#37d5f99a7f12c9ba90c4e2ac92e54ab6']

In [ ]:
# get the id of articles in popularity order
popularityArticles = [x[0] for x in popularityOrdered]

In [48]:
popularityArticles

['@brunch_151',
 '@sweetannie_145',
 '@intlovesong_28',
 '@tenbody_1305',
 '@conbus_43',
 '@chofang1_15',
 '@seochogirl_1',
 '@brunch_145',
 '@seochogirl_18',
 '@deckey1985_51',
 '@dailylife_207',
 '@steven_179',
 '@brunch_152',
 '@seochogirl_17',
 '@seochogirl_16',
 '@tenbody_1164',
 '@seochogirl_6',
 '@brunch_149',
 '@hyehyodam_19',
 '@hjl0520_26',
 '@shanghaiesther_46',
 '@seochogirl_7',
 '@seochogirl_8',
 '@noey_130',
 '@brunch_148',
 '@seochogirl_11',
 '@seochogirl_28',
 '@wootaiyoung_85',
 '@seochogirl_3',
 '@seochogirl_12',
 '@bzup_281',
 '@seochogirl_2',
 '@seochogirl_20',
 '@seochogirl_29',
 '@seochogirl_10',
 '@kam_60',
 '@seochogirl_13',
 '@seochogirl_14',
 '@brunch_141',
 '@seochogirl_5',
 '@seochogirl_9',
 '@seochogirl_15',
 '@mothertive_66',
 '@kotatsudiary_66',
 '@wikitree_54',
 '@dailylife_219',
 '@seochogirl_4',
 '@brunch_144',
 '@brunch_147',
 '@sangheeshyn_66',
 '@roysday_307',
 '@shindong_38',
 '@honeytip_945',
 '@mightysense_9',
 '@psychiatricnews_18',
 '@dong02_13

In [49]:
# make a dictionary of articles with ranks as their values
popularityRankDict = {k:v for v, k in enumerate(popularityArticles)}

In [50]:
popularityRankDict

{'@brunch_151': 0,
 '@sweetannie_145': 1,
 '@intlovesong_28': 2,
 '@tenbody_1305': 3,
 '@conbus_43': 4,
 '@chofang1_15': 5,
 '@seochogirl_1': 6,
 '@brunch_145': 7,
 '@seochogirl_18': 8,
 '@deckey1985_51': 9,
 '@dailylife_207': 10,
 '@steven_179': 11,
 '@brunch_152': 12,
 '@seochogirl_17': 13,
 '@seochogirl_16': 14,
 '@tenbody_1164': 15,
 '@seochogirl_6': 16,
 '@brunch_149': 17,
 '@hyehyodam_19': 18,
 '@hjl0520_26': 19,
 '@shanghaiesther_46': 20,
 '@seochogirl_7': 21,
 '@seochogirl_8': 22,
 '@noey_130': 23,
 '@brunch_148': 24,
 '@seochogirl_11': 25,
 '@seochogirl_28': 26,
 '@wootaiyoung_85': 27,
 '@seochogirl_3': 28,
 '@seochogirl_12': 29,
 '@bzup_281': 30,
 '@seochogirl_2': 31,
 '@seochogirl_20': 32,
 '@seochogirl_29': 33,
 '@seochogirl_10': 34,
 '@kam_60': 35,
 '@seochogirl_13': 36,
 '@seochogirl_14': 37,
 '@brunch_141': 38,
 '@seochogirl_5': 39,
 '@seochogirl_9': 40,
 '@seochogirl_15': 41,
 '@mothertive_66': 42,
 '@kotatsudiary_66': 43,
 '@wikitree_54': 44,
 '@dailylife_219': 45,
 '@

In [33]:
# defining global variables for analysis

recommRes = {}
detailForTestUsers = {}

maxFollowRate = 0.9 # 유사도 최대 배분 한계치 - 인기있는 글이 자동 추천될 수 있는 여지를 남겨둔 것
noveltyFullScore = 30 # 최신성 최대 점수
othersFullScore = 100 - noveltyFullScore

In [137]:
# 추천 결과 저장용 변수
res = open("saveRes.txt", 'w')

In [35]:
# 추천 생성부

for count, user in enumerate(testUsers) : 
    
    # 진행상황 확인
    print(count, user)
    
    # 과거 글을 읽은 기록이 있는지 확인
    try : 
        history = userwiseReadHistory[user]
    except KeyError : 
        history = []
    
    # 팔로잉하는 작가가 있는지 확인
    try : 
        followings = userFollowers[user]
    except KeyError : 
        followings = []
        
    # 각 독자별 구독 강도(읽은 글 중 팔로잉 작가의 글 비율) 및 독자별 제목 벡터 로드
    followRate = testUsersDict[user][0]
    readerVector = testUsersDict[user][-1]
    
    print("followRate : %f" % followRate)
    
    # 결과 저장용 리스트 생성
    recommRes[user] = []
    detailForTestUsers[user] = []
    
    # 과거 읽은 기록이 있는 경우 - 읽은 글은 다시 읽지 않을 것이라는 가정 하에 예비 목록에서 제거
    if len(history)> 0 :
        # 읽은 글의 인덱스 확인
        historyIndices = [titleDict[x] if x in titleDict else -1 for x in history]
        
        alreadyReadUniqueIndices = np.unique(historyIndices)
        alreadyReadUniqueIndices = alreadyReadUniqueIndices[np.where(alreadyReadUniqueIndices>=0)]
        
        # 읽지 않은 글만 남겨놓고 나머지 제거
        unreadArticles = metaDataForRecomm.drop(alreadyReadUniqueIndices)
        
        # 읽지 않은 글에 대하여 각 독자의 제목 벡터와 코사인 유사도 계산
        print("history exists; cosine similarity calculation")
        coss = titleTfidfMatrix.dot(readerVector.transpose()).toarray()
        
    # 과거 읽은 기록이 없는 경우 - 모든 글을 예비 목록으로 활용
    else : 
        unreadArticles = metaDataForRecomm
    
    # 팔로우하는 작가가 있는 경우 - 읽지 않은 글을 팔로우하는 작가의 글과 그밖의 글로 구분
    if len(followings) > 0 : 
        followingArticleFilter = np.isin(unreadArticles.writer, followings)
        unreadArticlesFromFollowings = unreadArticles[followingArticleFilter]
        unreadArticlesFromOthers = unreadArticles[~followingArticleFilter]
        
        if unreadArticlesFromFollowings.shape[0] :
            articleDfs = [unreadArticlesFromFollowings, unreadArticlesFromOthers]
        else : 
            articleDfs = [unreadArticlesFromOthers]
    
    # 팔로우하는 작가가 없는 경우 - 그대로 활용
    else : 
        articleDfs = [unreadArticles]
    
    # 팔로우하는 작가가 쓴 글에서 몇 개를 추천하고 그밖의 글에서 몇 개를 추천할지를 계산
    howManyFromFollowings = min(unreadArticlesFromFollowings.shape[0], round(followRate * 100), int(maxFollowRate*100))
    howManyFromOthers = 100-howManyFromFollowings
    
    howMany = [howManyFromFollowings, howManyFromOthers]
    
    # 유사도 및 인기도 점수 배분
    similarityFullScore = min(round(othersFullScore * followRate), othersFullScore*maxFollowRate)
    popularityFullScore = othersFullScore - similarityFullScore
    
    print(howManyFromFollowings, howManyFromOthers, similarityFullScore, popularityFullScore)
    
    
    print("start calculation")
    
    # 점수 계산 과정
    
    # 팔로우하는 작가가 있는 경우와 없는 경우를 구분하여 계산하기 위한 작업
    for idx, articleDf in enumerate(articleDfs) : 
        
        totalArticles = articleDf.shape[0]
        articlesNumpy = articleDf.articleId.values
        classType = "" # 팔로우하는 작가의 글은 0, 그밖의 글은 1
        
        # 팔로우하는 작가가 없는 경우
        if isinstance(articleDfs[0], int) : 
            classType = 1
        # 팔로우하는 작가가 있는 경우
        else : 
            if idx == 0 :
                classType = 0
            else : 
                classType = 1
        
        # 각 글별 점수 저장을 위한 딕셔너리
        scoreDict = dict(zip(articlesNumpy, [[classType, 0, 0, 0, 0] for x in range(totalArticles)]))
        
        #calculating novelty score
        print("novelty score")
        
        # 오래된 순서부터 글 정렬
        noveltySortedArticles = articleDf.sort_values('datetime').articleId
        # 등간격 점수 배분
        scoreList = np.linspace(0, noveltyFullScore, totalArticles)
        
        # 최신성 점수 저장
        for count, article in enumerate(noveltySortedArticles) : 
            scoreDict[article][1] = scoreList[count]
            
        #calculating similarity score only when history exists
        print("similarity score")
        
        if len(history) > 0 : # 읽은 기록이 있는 경우

            print('slicing coss') # 읽지 않은 글에 대해 계산해 둔 코사인 유사도 중 해당되는 글만의 값을 추려냄
            cossSliced = coss[articleDf.index]

            print('sorting') # 추려낸 값을 오름차순으로 정렬    
            similarSorted = sorted(zip(articlesNumpy, cossSliced), key = lambda x : x[1][0])
            
            # 유사도 점수 배분
            scoreList = np.linspace(0, similarityFullScore, totalArticles)
            
            print('giving similarity score')
            #유사도 점수 저장
            for count, articleAndScore in enumerate(similarSorted) : 
                scoreDict[articleAndScore[0]][2] = scoreList[count]
                
        # 인기도 점수 계산
        print("popularity score")
        # 읽힌 기록이 없어 인기도 자체가 계산되지 않은 경우 인기도 점수의 50%를 기본으로 부여 
        # - 주어진 로그 기록의 시간 범위를 벗어나 발행된 글이 있는 것에 대한 조치
        meanValue = popularityFullScore / 2
        
        # 읽힌 기록이 있는 것과 없는 것으로 구분
        popInfoExists = articleDf[[True if x in popularityRankDict else False for x in articlesNumpy]]
        popInfoNon = articleDf.drop(popInfoExists.index)
        
        print('sorting popularity')
        popInfoExistsSorted = sorted(popInfoExists.articleId, key=lambda x : popularityRankDict.get(x))
        
        # 인기도 점수 배분
        scoreList = np.linspace(0, popularityFullScore, len(popInfoExistsSorted))
        
        # 인기도 점수 저장
        for count, article in enumerate(popInfoExistsSorted) : 
            scoreDict[article][3] = scoreList[count]
            
        for article in popInfoNon.articleId : 
            scoreDict[article][3] = meanValue
            
        # summing all scores
        print('summing')
        totalSorted = []
        for article in scoreDict : 
            scoreDict[article][4] = sum(scoreDict[article][1:4])
        
        # 합산된 총점을 높은 순서대로 정렬
        totalSorted = sorted(scoreDict.items(), key = lambda x : x[1][4], reverse=True)
        
        print("assigning results")
        # 추천 결과 저장
        recommRes[user] += [x[0] for x in totalSorted[:howMany[idx]]]
        detailForTestUsers[user] += totalSorted[:howMany[idx]]

3250 #00f01d1d787d5288797ba2322a5c59a1
followRate : 0.400000
history exists; cosine similarity calculation
40 60 28 42
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3251 #865137a11ebaa067eabc83b4d8d7a96f
followRate : 0.743392
history exists; cosine similarity calculation
74 26 52 18
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3252 #37659f3d709409f3aa9fba71303f48c3
followRate : 0.000000
history exists; cosine similarity calculation
0 100 0 70
start ca

giving similarity score
popularity score
sorting popularity
summing
assigning results
3269 #097c1c194f69497fc26bfd439c084615
followRate : 0.728111
history exists; cosine similarity calculation
73 27 51 19
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3270 #b5f0c94fbb2a2fd7381a247c8dc8599f
followRate : 0.026087
history exists; cosine similarity calculation
3 97 2 68
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3271 #6888e3a79290f4b66c17155af463127f
fol

summing
assigning results
3288 #5f2f98d3d25da45b4c39b568a5416364
followRate : 0.258824
history exists; cosine similarity calculation
26 74 18 52
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3289 #5b0c99363edfed2d8ee51f45250ff849
followRate : 0.000000
history exists; cosine similarity calculation
0 100 0 70
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3290 #4863e55f85e7635d925f591e82987bc7
followRate : 0.507576
history exists; cosine similarity calcul

similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3307 #3c542092de96effe7b27157c55b55351
followRate : 0.033296
history exists; cosine similarity calculation
3 97 2 68
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3308 #bad980e76ce1ca668a57a02fe1b973fd
followRate : 0.095571
history exists; cosine similarity calculation
10 90 7 63
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3309 

assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3326 #990f01bb93545c4651cffd54f3b7f3e6
followRate : 0.303665
history exists; cosine similarity calculation
30 70 21 49
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3327 #f1429116afaf11ff5d4c30f86cf6694b
followRate : 0.583548
history exists; cosine similarity calculation
58 42 41 29
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sor

...

34 66 24 46
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3421 #1a32c64bbdd9bb6ed1f62b5a00d4950c
followRate : 0.444130
history exists; cosine similarity calculation
44 56 31 39
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3422 #5090dd88b6d3f8f754b608c85bf7d432
followRate : 0.155963
history exists; cosine similarity calculation
16 84 11 59
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
populari

87 13 61 9
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3440 #c65d7bc176069f0d92104fa690407f49
followRate : 0.714286
history exists; cosine similarity calculation
71 29 50 20
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3441 #f737a56b755a8a79747646a2b632da49
followRate : 0.213592
history exists; cosine similarity calculation
21 79 15 55
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularit

53 47 37 33
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3459 #7ba5ff376c52b00929c762d616636a62
followRate : 0.000000
history exists; cosine similarity calculation
0 100 0 70
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3460 #2b4ef54663bfba74b7645ed6d1c58e2b
followRate : 0.657143
history exists; cosine similarity calculation
66 34 46 24
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularit

novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3478 #1bb3bb8ab9302057fc50a3c101d9378e
followRate : 0.157282
history exists; cosine similarity calculation
16 84 11 59
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3479 #e3a9adb0720800c61b851fd38d189c1a
followRate : 0.035000
history exists; cosine similarity calculation
4 96 2 68
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assignin

2 98 1 69
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3497 #d66560626fa204868dacb73ed71f1415
followRate : 0.430149
history exists; cosine similarity calculation
43 57 30 40
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
articleDf 1
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity score
sorting popularity
summing
assigning results
3498 #2848ea8028085a8c87416f6048029e95
followRate : 0.000000
history exists; cosine similarity calculation
0 100 0 70
start calculation
articleDf 0
novelty score
similarity score
slicing coss
sorting
giving similarity score
popularity 

In [140]:
# 최종 추천 결과 저장
# - 당시 컴퓨팅 파워가 부족해 여러 컴퓨터에 나누어서 계산한 관계로 일단 결과물을 pickle로 받고 나중에 하나로 합칠 생각이었음

with open("recommRes.pkl", 'wb') as f:
    pickle.dump(recommRes, f)

with open('detailForTestUsers.pkl', 'wb') as f:
    pickle.dump(detailForTestUsers, f)

In [92]:
# 최종 결과 파일 생성부 - totalRecomm은 여러 컴퓨터에 나누어 계산한 결과물을 모두 합친 python dictionary였음

res1 = open('devuser/recommend.txt', 'w')
res2 = open('testuser/recommend.txt', 'w')

for user in testUsers : 
    eachResText = " ".join(totalRecomm[user])
    eachResText = user + " " + eachResText + '\n'
    
    res2.write(eachResText)
    if user in devUsers : 
        res1.write(eachResText)
        
res2.close()
res1.close()